# Wikipedia Analysis

### Context - page rank algorithm
The goal of this algorithm was to set a weight for each webpage on the internet, based on the number of external links pointing to the page.

### Context - goals of this project
We will be implementing the original algorithm to set the weights for each page in Wikipedia (English version), using the image stored in the Databricks Databases Set. This database contains 5823210 entries, stored in a parquet set of files.


#### Steps we will be following
- 1. Configuration of the library
- 2. Checking of the raw data structure
- 3. Extraction of relevant data
- 4. Creation of a table of forward links
- 5. Computation of the number of outgoing links
- 6. Creation of a table of reverse links 
- 7. Creation of the page rank table 
- 8. Definition of the number of iterations for the page rank


Extract relevant data: [Document ID, List of links]
Transform List of links to List od Docuemts ID: Forward Links Table
Calculate Number of output links
Construct Reverse Links Table from Forward Links Table
Initialize Page Rank Table
Recalculate Page Rank Table until:
All the Page Rank values are stable
Reach number of iterations (sugested value: 20 iterations)

# 1. Library configuration

In [0]:
import pandas as pd
import re
import math #new 

In [0]:
from pyspark.sql.types import *
from pyspark.sql.types import ArrayType, StringType,LongType
from pyspark.sql.functions import size, explode, collect_list 

# new
""" 
from pyspark.sql.types import ArrayType, StringType,LongType, FloatType
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from operator import add 
"""


Out[2]: ' \nfrom pyspark.sql.types import ArrayType, StringType,LongType, FloatType\nfrom pyspark.sql import functions as F\nfrom pyspark.sql import SparkSession\nfrom operator import add \n'

In [0]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true") # should it be true or false? 

# 2. Checking of the raw data structure
Now that our libraries are configured we will import the dataset and inspect it in its raw form in order to get an idea of the data that we will be handling.

In [0]:
# The URI for the database, which contains 5823210 entries, stored in a parquet set of files.
# We will now create a Spark DataFrame from those parquet files with the following command.

wikipediaDF = spark.read.parquet("dbfs:/databricks-datasets/wikipedia-datasets/data-001/en_wikipedia/articles-only-parquet")

As we can see, this database contains 7 columns with the following names and datatypes:
- title:string
- id:integer
- revisionId:integer
- revisionTimestamp:timestamp
- revisionUsername:string
- revisionUsernameId:integer
- text:string

It is important to be aware of which datatype each of this columns is storing, else our algorithm will not function correctly. 
The most important information for our algorithm is stored in the columns named “title”, “id”, and “text”.

In [0]:
# Since this command is not relevant for our algorithm and takes a long time to run (1.76 min), we decided to comment it out. 
# If we do run it we can appreciate how large the initial dataset is.  
N = wikipediaDF.count() # I am not very sure how to read the result

In [0]:
# Instead of using the full database, we will use a smaller version to analyse the structure, with 0.01% of records (approx. 582 records)
# We will select this small percentage of records randomly. In order to minimize random results, we will set a fixed seed value equal to 0

PartialWikipediaDF = wikipediaDF.sample(fraction=0.0001,seed=0).cache()

In [0]:
# Since this command is not relevant for our algorithm and takes an even longer time to run (7.88 min), we decided to comment it out. 
# If we do run it we can appreciate how much our dataset has diminished in comparison to the original one. 
PartialWikipediaDF.count()

Out[7]: 576

In [0]:
display(PartialWikipediaDF)

title id revisionId revisionTimestamp revisionUsername revisionUsernameId text Godzilla 11986 683608938 2015-10-01T10:45:31.000+0000 Mariomassone 8097292 {{About|the monster|the film franchise|Godzilla (franchise)|other uses}} {{redirect|ゴジラ|other uses of "Gojira"|Gojira (disambiguation)}} {{pp-move-indef|small=yes}}{{pp-semi-indef|small=yes}} <!-- This article was pared-down and heavily rewritten because it was a bloated with unsourced information and written in an inappropriate in-universe tone. Brevity is the soul of wit, please do not copy/paste information from Wikizilla or remove citations. --> {{Infobox character | colour = #001 | colour text = #ffa | name = Godzilla | series = [[Godzilla (franchise)|Godzilla film series]] | image = Godzilla '54 design.jpg | image_size = 320px | alt = | caption = Godzilla featured in the [[Godzilla (1954 film)|original film]] | first = ''[[Godzilla (1954 film)|Godzilla]]'' (1954) | last = ''[[Godzilla (2014 film)|Godzilla]]'' (2014) | creator = {{Plainlist| * [[Tomoyuki Tanaka]] * [[Ishirō Honda]] * [[Eiji Tsubaraya]] }} | portrayer = {{Plainlist| * '''Shōwa series''': * [[Haruo Nakajima]] {{cite video|people=[[Takeo Murata]] (writer) and [[Ishirō Honda]] (writer/director)|title=''[[Godzilla (1954 film)|Godzilla]]''|medium=[[DVD]]|publisher=[[Classic Media]]|date=2006}} {{cite video|people=[[Al C. Ward]] (writer) and [[Ishirō Honda]], [[Terry Morse]] (writers/directors)|title=''[[Godzilla, King of the Monsters!]]''|medium=[[DVD]]|publisher=[[Classic Media]]|date=2006}} * [[Katsumi Tezuka]] * [[Yū Sekida]] * Ryosaku Takasugi * [[Seiji Onaka]] * Shinji Takagi * Isao Zushi * [[Toru Kawai]] * '''Heisei series''': * [[Kenpachiro Satsuma]] * '''Millennium series''': * [[Tsutomu Kitagawa]] * [[Mizuho Yoshida (actor)|Mizuho Yoshida]] * '''TriStar Pictures''': * Kurt Carley {{cite web|url=http://www.tohokingdom.com/articles/2014-11-07_nakajima_carley_godzilla_1954_1998.html|title=Nakajima and Carley: Godzilla's 1954 and 1998|last=Mirjahangir|first=Chris|work=Toho Kingdom|date=November 7, 2014|accessdate=April 5, 2015}} * '''Legendary Pictures''': * TJ Storm {{cite web|url=http://translate.google.com/translate?hl=en&sl=es&u=http://www.crhoy.com/conozca-al-actor-que-da-vida-a-godzilla-quien-hablo-con-crhoy-com-v2k0k2x/&prev=search|title= Meet the actor who gives life to Godzilla|last=Arce|first=Sergio|work=crhoy.com|date=May 29, 2014|accessdate=March 26, 2015}} }} | lbl1 = Designed by | data1 = {{Plainlist| * [[Akira Watanabe (film art director)|Akira Watanabe]] * [[Teizô Toshimitsu]] Classic Media Godzilla 2006 DVD - [http://www.youtube.com/watch?v=yfSARjZ0OXc Special Features: Making of The Godzilla Suit] - Ed Godziszewski }} | alias = {{Plainlist| * King of the Monsters ''[[Godzilla, King of the Monsters!]]'' (1956). Directed by [[Ishirō Honda]] and [[Terry O. Morse]]. [[Toho]] ''[[Godzilla vs. Megaguirus]]'' (2000). Directed by Masaaki Tezuka. [[Toho]] ''[[Godzilla: Final Wars]]'' (2004). Directed by [[Ryuhei Kitamura]]. [[Toho]] {{cite web |url=http://www.comics.org/series/2392/covers |title=GCD :: Covers :: Godzilla |publisher=Comics.org |accessdate=October 18, 2011}} * Gigantis ''[[Godzilla Raids Again#English version|Godzilla Raids Again]]'' (US release) 1959 [[Toho]] * Monster Zero-One ''[[Invasion of Astro-Monster]]'' (1965). Directed by [[Ishirō Honda]]. [[Toho]] * God of Destruction ''[https://www.youtube.com/watch?v=ohbMHoX2B48 Godzila, Mothra, and King Ghidorah]'' (2001). Directed by [[Shusuke Kaneko]]. [[Toho]] {{cite web |url=http://www.scifijapan.com/articles/2010/07/04/godzilla-perfect-collection-box-6/ |title=GODZILLA SOUNDTRACK PERFECT COLLECTION BOX 6|last=DeSentis|first=John|work=Scifi Japan|accessdate=November 23, 2014}} * Alpha Predator ''[[Godzilla (2014 film)|Godzilla]]'' (2014). Directed by [[Gareth Edwards (director)|Gareth Edwards]]. [[Legendary Pictures]] }} | species = [[Kaiju|Daikaiju]] }} {{nihongo|'''Godzilla'''|ゴジラ|Gojira}} ({{IPAc-en|ɡ|ɒ|d|ˈ|z|ɪ|l|ə}}; {{IPA-ja|

Output can only be rendered in Databricks

# 3. Extraction of relevant data
Now that we have a rough idea of our dataset and have decreased its size to a more manageable subset, we will begin with our analysis.

As we mentioned previously and highlighted when displayed as a table, the most important information for our algorithm is stored in the columns named “title”, “id”, and “text”. 
- Title stores the title of the document (string)
- ID stores the ID of the document (integer)
- Text stores the content of the document (string)

This is intuitive when thinking about the purpose of the page rank algorithm, which is to rank web pages in their search engine results. In order to complete this task, the algorithm will need to know the name (title) of the page, what the page is about (text), and the unique identity of the page to distinguish it from others (ID). The most important piece of information for this algorithm is are the links that the page content includes, as the rating of importance each page will recieve is a recursively defined measure whereby a page becomes important if important pages link to it.

Thus, in this section we will focus on those three columns, and extract the key information from the text column, namely the links, which are identifiable as strings in brackets.

In [0]:
def parse_links(document_body):
  data=re.findall(r'\[\[(.+?)\]\]',document_body)
  if (len(data)>0):
    links=[s.lower() for s in data]
  else:
    links=[]
  return links

In [0]:
test="{{Use Indian English|date=April 2015}} {{Infobox person | name = Shavez Khan | image = | caption = | birth_date = | birth_place = India | nationality = India | residence = [[Mumbai]], India | occupation = [[Actor]] | years_active = present | height = }} '''Shavez Khan''' is an [[India]]n television [[actor]]. He has done his roles in various Indian television shows like Shaitaan,<ref>{{cite web|url=http://www.tellychakkar.com/tv/tv-news/shavez-khan-feature-episodic-of-colors-shaitaan|title=Shavez Khan to feature in an episodic of Colors' Shaitaan|work=Tellychakkar|date=11 April 2013|accessdate=24 April 2015}}</ref> [[Encounter (Indian TV series)|Encounter]], [[Ek Hasina Thi (TV series)|Ek Hasina Thi]], [[Savdhaan India]],<ref>{{cite web|url=http://www.tellychakkar.com/tv/tv-news/shavez-khan-anshul-singh-and-damini-joshi-episodic-of-savdhan-india-140915|title=Shavez Khan, Anshul Singh and Damini Joshi in an episodic of Savdhan India|work=Tellychakkar|date=15 September 2014|accessdate=24 April 2015}}</ref> [[SuperCops vs Supervillains]],<ref>{{cite web|url=http://www.tellychakkar.com/tv/tv-news/rituraj-singh-and-shavez-khan-life-oks-shapath-141009|title=Rituraj Singh and Shavez Khan in Life OK's Shapath|work=Tellychakkar|date=9 October 2014|accessdate=24 April 2015}}</ref> Pyaar Ka The End,<ref>{{cite web|url=http://www.tellychakkar.com/tv/tv-news/shavez-khan-bindass-pyaar-ka-the-end-141029|title=Shavez Khan in Bindass' Pyaar Ka The End|work=Tellychakkar|date=29 October 2014|accessdate=24 April 2015}}</ref> [[Pyaar Kii Ye Ek Kahaani]], [[MTV Fanaah]], [[Crime Patrol (TV series)|Crime Patrol]]. He has played his recent role in [[Sony Entertainment Television (India)|Sony TV]]'s [[C.I.D. (Indian TV series)|CID]].<ref>{{cite web|url=http://www.tellychakkar.com/tv/tv-news/shavez-khan-sony-tvs-cid-150417|title=Shavez Khan in Sony TV's CID|work=Tellychakkar|date=17 April 2015|accessdate=24 April 2015}}</ref> ==Television== *[[Colors (TV channel)|Colors]]'s Shaitaan *[[Sony Entertainment Television (India)|Sony TV]]'s [[Encounter (Indian TV series)|Encounter]], [[Crime Patrol (TV series)|Crime Patrol]] & [[C.I.D. (Indian TV series)|CID]] *[[Star Plus]]'s [[Ek Hasina Thi (TV series)|Ek Hasina Thi]] *[[Life OK]]'s [[Savdhaan India]] & [[SuperCops vs Supervillains]] *[[Bindass]]' Pyaar Ka The End *[[Star One]]'s [[Pyaar Kii Ye Ek Kahaani]] *[[MTV]]'s [[MTV Fanaah]] ==References== {{Reflist}} ==External links== {{Persondata | NAME = Khan, Shavez | ALTERNATIVE NAMES = | SHORT DESCRIPTION = Indian model and television actor | DATE OF BIRTH = <!--Birth date has been contested. Do not add without providing a reliably published source with a reputation for editorial oversight--> | PLACE OF BIRTH = India | DATE OF DEATH = | PLACE OF DEATH = }} {{DEFAULTSORT:Khan, Shavez}} [[Category:Living people]] [[Category:Indian male television actors]] [[Category:Actors in Hindi television]] [[Category:Indian television personalities]]"

In [0]:
parse_links(test)

Out[11]: ['mumbai',
 'actor',
 'india',
 'actor',
 'encounter (indian tv series)|encounter',
 'ek hasina thi (tv series)|ek hasina thi',
 'savdhaan india',
 'supercops vs supervillains',
 'pyaar kii ye ek kahaani',
 'mtv fanaah',
 'crime patrol (tv series)|crime patrol',
 'sony entertainment television (india)|sony tv',
 'c.i.d. (indian tv series)|cid',
 'colors (tv channel)|colors',
 'sony entertainment television (india)|sony tv',
 'encounter (indian tv series)|encounter',
 'crime patrol (tv series)|crime patrol',
 'c.i.d. (indian tv series)|cid',
 'star plus',
 'ek hasina thi (tv series)|ek hasina thi',
 'life ok',
 'savdhaan india',
 'supercops vs supervillains',
 'bindass',
 'star one',
 'pyaar kii ye ek kahaani',
 'mtv',
 'mtv fanaah',
 'category:living people',
 'category:indian male television actors',
 'category:actors in hindi television',
 'category:indian television personalities']

In [0]:
parse_links_udf = udf(parse_links,ArrayType(StringType()))

In [0]:
# Convert the text to lowercase 
tolower_udf = udf(lambda x: x.lower())

In [0]:
# decide if we keep it here or at the beginning of the code 
from pyspark.sql.functions import *

In [0]:
# Create a dataframe with the selected information and renaming title to title and text to links
dataDF1 = PartialWikipediaDF.select(tolower_udf("title").alias("title"),"id",parse_links_udf("text").alias("links")) # not sure if it should be "lower" or "tolower_udf" - think the second option

# 4. Creation of a table of forward links
Now we will transform our list of links to a list of document IDs. In order to extract the ID of the target linked documents (the pages that are linked) we will have to analyse the entire Wikipedia database and create a table with these two pieces of information.

In [0]:
# Extract the data from the full Wikipedia DF
title_idDF = wikipediaDF.select("id",tolower_udf("title").alias("title"));

In [0]:
# Convert this extraction to a pandas DF 
title_idPDF = title_idDF.toPandas()

In [0]:
TempForwardDF = PartialWikipediaDF.select("title","id",parse_links_udf("text").alias("links"))

In [0]:
display(TempForwardDF)

title id links Godzilla 11986 List(godzilla (franchise)|godzilla film series, godzilla (1954 film)|original film, godzilla (1954 film)|godzilla, godzilla (2014 film)|godzilla, tomoyuki tanaka, ishirō honda, eiji tsubaraya, haruo nakajima, takeo murata, ishirō honda, godzilla (1954 film)|godzilla, dvd, classic media, al c. ward, ishirō honda, terry morse, godzilla, king of the monsters!, dvd, classic media, katsumi tezuka, yū sekida, seiji onaka, toru kawai, kenpachiro satsuma, tsutomu kitagawa, mizuho yoshida (actor)|mizuho yoshida, akira watanabe (film art director)|akira watanabe, teizô toshimitsu, godzilla, king of the monsters!, ishirō honda, terry o. morse, toho, godzilla vs. megaguirus, toho, godzilla: final wars, ryuhei kitamura, toho, godzilla raids again#english version|godzilla raids again, toho, invasion of astro-monster, ishirō honda, toho, shusuke kaneko, toho, godzilla (2014 film)|godzilla, gareth edwards (director)|gareth edwards, legendary pictures, kaiju|daikaiju, kaiju|daikaiju, tokusatsu, godzilla (franchise)|films of the same name, japan, ishirō honda, godzilla (1954 film)|godzilla, godzilla video games|video games, godzilla (comics)|comic books, godzilla (franchise)#television|television shows, toho, godzilla_(franchise)#american_films|two hollywood films, godzilla, king of the monsters!, nuclear bombings of hiroshima and nagasaki, daigo fukuryū maru|lucky dragon 5, nuclear weapon, portmanteau, gorilla, whale, ateji, kanji, hepburn romanization, kunrei romanization, file:final wars godzilla.jpg|thumb|left|230px|godzilla's iconic design (seen here in ''[[godzilla: final wars, ray harryhausen, the beast from 20,000 fathoms, amphibian|amphibious, dinosaur, wired (magazine)|wired, tyrannosaurus, iguanodon, stegosaurus, alligator, usa today, chimera (mythology)|chimera, life (magazine)|''life'' magazine, keloid, hibakusha|hiroshima's survivors, turner classic movies, akira ifukube, double bass|contrabass, godzilla 2000, file:godzilla king of the monsters (1956) atomic ray.png|thumb|right|230px|godzilla's atomic breath, as shown in ''[[godzilla (1954 film)|godzilla, black belt (martial arts)|black belt, godzilla raids again, transitional form, regeneration (biology)|regenerate, godzilla 2000, takao okawara, toho, godzilla vs. king ghidorah, kazuki ōmori, toho, godzilla vs. mechagodzilla, jun fukuda, toho, godzilla vs. biollante, kazuki ōmori, toho, godzilla: destroy all monsters melee, pipeworks software, zone fighter, the godzilla power hour, godzilla vs. hedorah, yoshimitsu banno, toho, ghidorah, the three-headed monster, ishirō honda, toho, godzilla: unleashed, the return of godzilla, koji hashimoto (director)|koji hashimoto, toho, shinto, file:kk v g trailer (1962).png|thumb|left|godzilla battles [[king kong, king kong vs. godzilla, gender-neutral pronoun, godzilla (1998 film)|1998 godzilla film, parthogenesis, jsdf, king ghidorah, gigan, mechagodzilla, megalon, biollante, megaguirus, mothra, rodan, anguirus, minilla, king kong, fantastic four, godzilla (1954 film)|the original 1954 film, godzilla, king of the monsters!|1956 american version, joseph e. levine, tokyo metropolitan government building, godzilla vs. king ghidorah, legendary pictures|legendary, godzilla (2014 film)|godzilla, gareth edwards (director)|gareth edwards, godzilla (2016 film)|upcoming toho godzilla film, file:godzilla raids again (1955) behind the scenes.gif|thumb|suit fitting on the set of ''[[godzilla raids again, suitmation|an actor wearing a latex costume, animatronics|animatronic, stop-motion, computer generated imagery|computer-generated, king kong (1933 film)|king kong, eiji tsuburaya, the return of godzilla, godzilla vs biollante, godzilla vs destoroyah, kenpachiro satsuma, oxygen deprivation, godzilla vs. spacegodzilla, patrick tatopoulos, motion capture, digitigrade, computer-generated imagery|cgi, stan winston, jurassic park (film)|jurassic park, the moving picture company, bear, komodo dragon, lizard, lion, gray wolf|wolves

In [0]:
def titles2id(links,titleidPDF):
  data_titles=titleidPDF
  if (len(links)>0):
    ids=data_titles[data_titles.title.isin(links)].id.to_list()
  else:
    ids=[]
  return list(set(ids))

In [0]:
titles2id_UDF = udf(lambda x: titles2id(x,title_idPDF),ArrayType(LongType(),False))

In [0]:
ForwardDF = TempForwardDF.select("id",titles2id_UDF("links").alias("links")).cache()

In [0]:
display(ForwardDF)

id,links
11986,"List(36865, 303628, 79887, 36896, 930853, 20518, 1670693, 925736, 3135027, 567874, 208463, 634965, 18653283, 18637926, 267370, 17229419, 28272, 4627056, 8311, 1236610, 1862287, 1009817, 396968, 1236650, 53430, 302275, 870601, 11988, 15573, 5220058, 325852, 12000, 37604, 12005, 45178603, 9073903, 325875, 3572981, 9182971, 1909502, 9322751, 12546, 30467, 18993927, 18184, 226575, 192790, 147734, 1646358, 21785, 7396632, 23188764, 18057502, 1112869, 2120494, 4400, 105776, 3921, 162143, 11014498, 894309, 3540339, 1245556, 791422, 23624578, 11778948, 11664, 8660880, 570258, 570263, 1356696, 1790878, 13729, 18637733, 570279, 16823212, 248760, 228798, 30863297, 2156483, 17360, 30683, 266204, 20002271, 59361, 5765607, 65001, 9182702, 33777)"
13049,"List(42368, 230019, 6532, 2427653, 228998, 29604227, 94097, 12785172, 34543638, 323481, 199961, 208411, 21018, 248859, 32798, 670749, 383648, 15782, 18622119, 3038250, 70188, 213936, 47923, 147635, 23953851, 700, 875196, 27480261, 1550022, 87240, 1972169, 189774, 153295, 7997906, 22928212, 26202, 213978, 1047259, 803804, 44766, 310113, 946273, 209892, 39658852, 32742, 12521, 1129578, 5178089, 43245, 1056621, 16320493, 19444, 34115577, 161275, 3745788, 26255613)"
38010,"List(43638795, 18957, 11024, 30206738, 22303, 460321, 3117601, 18994087, 291242, 49838, 23868856, 13256, 42480973, 459471, 200033, 9955, 44905, 294126, 18209535)"
38252,"List(43008, 57858, 259082, 48147, 2000410, 156699, 10277, 5166, 2776113, 51763, 22068, 2555445, 72758, 2416701, 21611071, 2465857, 273476, 431685, 198217, 186444, 273484, 273486, 273487, 431695, 273489, 18949200, 30866008, 44635, 22113, 4883556, 152171, 191086, 915566, 18933360, 43956336, 13696118, 21113, 238715, 9282173, 19344515, 31365, 42634, 8803981, 148113, 1775761, 28309, 2416795, 23648411, 303261, 146595, 22944933, 79026, 1721523, 25804468, 462002, 25817785, 228538, 18994363, 437949, 182462, 5369534, 16579, 591556, 232142, 639185, 92374, 1574616, 24061144, 31453, 2004720, 30456, 30419193, 5126908, 15613, 19698439, 10510, 19344654, 53518, 170772, 170773, 205078, 954140, 2041117, 22308, 7129894, 9510, 47777069, 355118, 51503, 6974, 172350, 28480, 1732421, 29511, 6766412, 13825869, 2798927, 140624, 140630, 1462102, 238424, 7038301, 34041182, 719715, 235879, 136566, 1029495, 1654648, 170874, 77179, 1121666, 156547, 17292, 1529230, 202652, 20110751, 1712034, 30860202, 45294511, 416688, 17339824, 2632114, 20405, 276919, 16728504, 1560005, 2291656, 4865481, 8642505, 2520013, 319442, 5079506, 928738, 1714150, 24047593, 13603306, 27424748, 805870, 167409, 1571314, 1040371, 7668, 32722929, 33642486, 13695994, 2808829)"
196900,"List(198400, 51847, 51848, 73100, 161935, 3985, 50585, 68145, 199608, 324, 44748, 26956, 180432, 198354, 42524885, 201174, 140632, 197469, 50287, 7672)"
201227,"List(400064, 26787, 73257, 146730, 501161, 31756, 11054, 102446, 382576, 24113, 3434750, 2764750, 21780, 24850, 350712, 50714, 32190, 2731583)"
324799,"List(5034215, 728168, 15807, 1121587, 728596, 14838867, 1287577, 47422, 11615)"
162369,"List(2012485, 21255, 21259, 5345872, 15572722, 154099, 392436, 695507, 19718837, 25084215, 7380029)"
340927,"List(3414021, 340934, 153223, 263163, 553623, 151210, 2481352, 4980140, 25391, 3434750, 340947, 22309268, 340917, 869878, 357847, 31769, 26779, 489179)"
343031,"List(166404, 19604228, 251399, 31880, 30862736, 14865, 4116, 318742, 610583, 5916, 67229, 144417, 61478, 2406953, 1536565, 59958, 285623, 224698, 728892, 23874378, 38992, 1147994, 736, 9697, 147939, 10363747, 18399589, 563689, 12024, 10489)"


# 5. Computation of the number of outgoing links
Now we will compute the number of outgoing links for each page (document). This is a crucial step because our page rank algorithm needs this information in order to adequately perfom its ranking and importance calculations.

In [0]:
# Compute the number of outgoing links for each document
OutgoingsLinksCountersDF = ForwardDF.select("id",size("links").alias("counter"))

In [0]:
display(OutgoingsLinksCountersDF)

# 6. Creation of a table of reverse links

In [0]:
TemporalReverseLinks = ForwardDF.select("id",explode("links").alias("t_link"))

In [0]:
TemporalReverseLinks.show(10)

In [0]:
ReverseDF = TemporalReverseLinks.groupBy("t_link").agg(collect_list ("id").alias("links")).cache()

In [0]:
display(ReverseDF)

# 7. Creation of the page rank table

In [0]:
pageRankPDF = ReverseDF.select("t_link").toPandas()

In [0]:
pageRankPDF["PR"] = 0.85/N #N is the total number of documents in the Wikipedia 2015

In [0]:
display(pageRankPDF)

# 8. Definition of the number of iterations for the page rank
Now we need to determine how many times the page rank should calculate the rank. To do this, we will use a loop.

In [0]:
def new_pagerank(links, current_pr, counters):
  n_pr = 0;
  for l in links:
    #get_ current_pr
    #get counter
    n_pr += current_pr/counter
  new_pr = 0.85/N+0.15*n_pr
  return new_pr

In [0]:
new_pagerank_udf = udf (lambda (x,y): new_pagerank(x,pagerankPDF,y), DoubleType())

In [0]:
# WHILE LOOP WITH SEVERAL EXIT CONDITIONS
new_pagerank_udf = udf (lambda (x,y): new_pagerank(x,pagerankPDF,y), DoubleType())
NewPageRankDF=ReverseDF.select(ReverseDF["t_link"],new_page_rank_udf(ReverseDF["links"],ReverseDF["counters"]).alias("PR"))
pagerankPDF=NewPageRankDF.toPandas()

# Conclusion